# Pandas的数据结构



导入pandas：  
三剑客

In [6]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

### 1、Series

Series是一种类似与一维数组的对象，由下面两个部分组成：
- values：一组数据（ndarray类型）
- index：相关的数据索引标签

#### 1）Series的创建

两种创建方式：

(1) 由列表或numpy数组创建

    默认索引为0到N-1的整数型索引

In [5]:
#使用列表创建Series
Series(data=[1,2,3,4],index=['ds','dsa','re','gr'],name='haha')

ds     1
dsa    2
re     3
gr     4
Name: haha, dtype: int64

In [9]:
#使用numpy创建Series
Series(data=np.arange(10,60,6))

0    10
1    16
2    22
3    28
4    34
5    40
6    46
7    52
8    58
dtype: int32

In [4]:
#使用name参数


- 还可以通过设置index参数指定索引

(2) 由字典创建:不能在使用index.但是依然存在默认索引

注意：数据源必须为一维数据

In [13]:
dic = {
    'math':100,
    'English':50
}
Series(data=dic,name='qimo')

math       100
English     50
Name: qimo, dtype: int64

============================================

练习1：

使用多种方法创建以下Series，命名为s1：  
语文 150   
数学 150   
英语 150   
理综 300   

============================================

#### 2）Series的索引和切片

可以使用中括号取单个索引（此时返回的是元素类型），或者中括号里<font color=red>一个列表</font>取多个索引（此时返回的是一个Series类型）。

(1) 显式索引：

    - 使用index中的元素作为索引值
    - 使用s.loc[]（推荐）:注意，loc中括号中放置的一定是显示索引

 注意，此时是闭区间

In [14]:
s = Series(np.random.randint(60,100,size=(5,)),index=['a','b','c','d','e'])
s

a    85
b    65
c    64
d    99
e    93
dtype: int32

In [16]:
s['b']

65

(2) 隐式索引：

    - 使用整数作为索引值
    - 使用.iloc[]（推荐）:iloc中的中括号中必须放置隐式索引

 注意，此时是半开区间

In [20]:
s.iloc[1]

65

In [21]:
s.iloc[[1,2,3]]

b    65
c    64
d    99
dtype: int32

## 切片:隐式索引切片和显示索引切片

- 显示索引切片:index和loc

In [18]:
s.loc['b']

65

- 隐式索引切片：整数索引值和iloc

In [22]:
s.loc['a':'c']

a    85
b    65
c    64
dtype: int32

#### 3）Series的基本概念

可以把Series看成一个定长的有序字典

向Series增加一行：相当于给字典增加一组键值对

In [26]:
s['f'] = 100
s

a     85
b     65
c     64
d     99
e     93
f    100
dtype: int64

可以通过shape，size，index,values等得到series的属性

In [27]:
s.index

Index(['a', 'b', 'c', 'd', 'e', 'f'], dtype='object')

In [28]:
s.values

array([ 85,  65,  64,  99,  93, 100], dtype=int64)

可以使用s.head(),tail()分别查看前n个和后n个值

In [29]:
s.head(3)

a    85
b    65
c    64
dtype: int64

对Series元素进行去重

In [31]:
s = Series(data=[1,1,2,2,3,3,4,4,4,5,6,7,8,7,7,66,43,342,6665,444,333,444])
s.unique()   #Series中的去重函数

array([   1,    2,    3,    4,    5,    6,    7,    8,   66,   43,  342,
       6665,  444,  333], dtype=int64)

当索引没有对应的值时，可能出现缺失数据显示NaN（not a number）的情况

- 使得两个Series进行相加

In [32]:
s1 = Series(data=[1,2,3,4,5],index=['a','b','c','d','e'])
s2 = Series(data=[1,2,3,4,5],index=['a','b','c','f','g'])

In [35]:
s = s1+s2
s

a    2.0
b    4.0
c    6.0
d    NaN
e    NaN
f    NaN
g    NaN
dtype: float64

可以使用pd.isnull()，pd.notnull()，或s.isnull(),notnull()函数检测缺失数据

In [40]:
s.iloc[[True,False,True,True,False,True,True]]   #True/False是可以作为Series的索引

a    2.0
c    6.0
d    NaN
f    NaN
g    NaN
dtype: float64

In [43]:
s.loc[s.notnull()]   #空值检测,过滤

a    2.0
b    4.0
c    6.0
dtype: float64

In [44]:
s

a    2.0
b    4.0
c    6.0
d    NaN
e    NaN
f    NaN
g    NaN
dtype: float64

In [45]:
np.__version__

'1.15.1'

#### 4）Series的运算

(1) + - * /

(2) add() sub() mul() div()  :  s1.add(s2,fill_value=0)

(3) Series之间的运算

- 在运算中自动对齐不同索引的数据
- 如果索引不对应，则补NaN

============================================

练习3：

1. 想一想Series运算和ndarray运算的规则有什么不同？

2. 新建另一个索引包含“文综”的Series s2，并与s2进行多种算术操作。

============================================

### 2、DataFrame

DataFrame是一个【表格型】的数据结构。DataFrame由按一定顺序排列的多列数据组成。设计初衷是将Series的使用场景从一维拓展到多维。DataFrame既有行索引，也有列索引。
- 行索引：index
- 列索引：columns
- 值：values

#### 1）DataFrame的创建
最常用的方法是传递一个字典来创建。DataFrame以字典的键作为每一【列】的名称，以字典的值（一个数组）作为每一列。

此外，DataFrame会自动加上每一行的索引。

使用字典创建的DataFrame后，则columns参数将不可被使用。

同Series一样，若传入的列与字典的键不匹配，则相应的值为NaN。


In [49]:
DataFrame(data=np.random.randint(60,100,size=(3,3)),index=['a','b','c'],columns=['A','B','C'])

,A,B,C
a,99,69,99
b,91,73,75
c,64,94,74


- 使用ndarray创建DataFrame

In [52]:
df = DataFrame(data=np.random.randint(60,100,size=(3,3)),index=['a','b','c'],columns=['A','B','C'])
df

,A,B,C
a,63,90,60
b,60,91,97
c,82,73,83


DataFrame属性：values、columns、index、shape

In [53]:
df.values

array([[63, 90, 60],
       [60, 91, 97],
       [82, 73, 83]])

In [54]:
df.index

Index(['a', 'b', 'c'], dtype='object')

In [55]:
df.columns

Index(['A', 'B', 'C'], dtype='object')

使用ndarray创建DataFrame：创建一个表格用于展示张三，李四，王五的java，python的成绩

============================================

练习4：

根据以下考试成绩表，创建一个DataFrame，命名为df：
```
    张三  李四  
语文 150  0
数学 150  0
英语 150  0
理综 300  0
```

============================================

In [57]:
df = DataFrame(data=np.random.randint(60,150,size=(4,2)),index=['chinese','math','english','lizong'],columns=['zhangsan','li'])
df

,zhangsan,li
chinese,89,88
math,68,89
english,69,101
lizong,120,102


In [61]:
dic = {
    'java':[60,70,80],
    'python':[100,100,100]
}
DataFrame(data=dic,index=['zhangsan','lisi','wangwu'])

,java,python
zhangsan,60,100
lisi,70,100
wangwu,80,100


#### 2）DataFrame的索引

(1) 对列进行索引

    - 通过类似字典的方式  df['q']
    - 通过属性的方式     df.q

 可以将DataFrame的列获取为一个Series。返回的Series拥有原DataFrame相同的索引，且name属性也已经设置好了，就是相应的列名。

In [5]:
#修改列索引


In [6]:
#获取前两列


(2) 对行进行索引

    - 使用.loc[]加index来进行行索引
    - 使用.iloc[]加整数来进行行索引
    
 同样返回一个Series，index为原来的columns。

(3) 对元素索引的方法
    - 使用列索引
    - 使用行索引(iloc[3,1] or loc['C','q']) 行索引在前，列索引在后
    

#### 切片：

【注意】
直接用中括号时：
- 索引表示的是列索引
- 切片表示的是行切片

在loc和iloc中使用切片(切列) ：      df.loc['B':'C','丙':'丁']

============================================

练习5：

使用多种方法对ddd进行索引和切片，并比较其中的区别

============================================

In [62]:
import tushare

#### 3）DataFrame的运算

（1） DataFrame之间的运算

同Series一样：

- 在运算中自动对齐不同索引的数据
- 如果索引不对应，则补NaN

创建DataFrame df1 不同人员的各科目成绩，月考一

创建DataFrame df2 不同人员的各科目成绩，月考二  
有新学生转入

============================================

练习6：

1. 假设ddd是期中考试成绩，ddd2是期末考试成绩，请自由创建ddd2，并将其与ddd相加，求期中期末平均值。

2. 假设张三期中考试数学被发现作弊，要记为0分，如何实现？

3. 李四因为举报张三作弊立功，期中考试所有科目加100分，如何实现？

4. 后来老师发现有一道题出错了，为了安抚学生情绪，给每位学生每个科目都加10分，如何实现？

============================================